In [1]:
from runtest import *
from Data_binary import *
from statistics import median

In [2]:
env = Data(unknown_rate=1)
env.loadfile("hcv.csv") # change this to the test file
env.normalize()
env.alpha = 0
env.cluster_K_means(7)
# this makes it so when we do the ranking, we only check against this number
# of data points; ignore for now
#env.set_validation(2000)
# makes the costs uniform; we won't have groups either
env.set_costs()

# partition into training/test sets
test_env = env.split(0.80)
print("training points:", len(env.data))
print("test points:", len(test_env.data))

env.write_data("../tree/hcv_train.csv")
test_env.write_data("../tree/hcv_test.csv")

training points: 493
test points: 122


In [ ]:
costs = [(i+1)/13 for i in range(13)] # list of maximum budgets
results = [] # stores all the results for each value of budget
for c in costs:
    env.max_cost = c
    test_env.max_cost = c
    # take the average of 3 trainings
    r1 = []
    r2 = []
    for i in range(3):
        rtemp = runtest(env,test_env)
        r1.append(rtemp[2])
        r2.append(rtemp[3])
    print("At cost:", c, median(r1), median(r2))
    results.append([r1,r2])

In [ ]:
import matplotlib.pyplot as plt

# this value is the total distance between p and 5 closest predicted points to p
resRL = [median(results[i][0]) for i in range(len(results))]
resRAND = [median(results[i][1]) for i in range(len(results))]
plt.plot(costs, resRL, 'r', costs, resRAND)
plt.show()

In [ ]:
# we will write the results to BENCHMARK/<test>.csv
f = open('BENCHMARK/hcv_test.csv', 'w')

def write(f,lst):
    for l in lst:
        f.write(str(l))
        f.write(' ')

# first write the number of test points
f.write(str(len(test_env.data)))
f.write('\n')
        
# write the cost
write(f,costs)
f.write('\n')

# write the RL result
write(f,resRL)
f.write('\n')

# finally write the random results
write(f, resRAND)
f.write('\n')

f.close()